In [78]:
!pip install pandas openpyxl
!pip install holidays tensorflow xlrd
import pandas as pd
import numpy as np
import holidays
import os
import datetime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from datetime import datetime, timedelta
from tensorflow.keras.layers import Input

df_list = []

for year in range(2004, 2025):
  file_name = f'austin, texas {year}-01-01 to {year}-12-31.xlsx'
  df_year = pd.read_excel(file_name)
  df_year.rename(columns={df_year.columns[0]: 'timestamp'}, inplace=True)
  df_year['timestamp'] = pd.to_datetime(df_year['timestamp'])
  df_year['timestamp'] = df_year['timestamp'].dt.round('h')

  # create the hour, day of week, month, and is_weekend
  df_year['hour'] = df_year['timestamp'].dt.hour
  df_year['day'] = df_year['timestamp'].dt.day
  df_year['year'] = df_year['timestamp'].dt.year
  df_year['month'] = df_year['timestamp'].dt.month
  # drop daylight savings dupes
  df_year = df_year[~df_year['timestamp'].duplicated()]
  # if there are any gaps in the data
  df_year['temp'] = df_year['temp'].ffill()
  df_year['humidity'] = df_year['humidity'].ffill()
  df_year['windspeed'] = df_year['windspeed'].ffill()
  df_year['cloudcover'] = df_year['cloudcover'].ffill()

  df_list.append(df_year)

df = pd.concat(df_list, ignore_index=True)
print("Combined shape:", df.shape)


temp_list = []
temp_df = pd.read_excel("ghi_san_antonio.xlsx")
temp_df = temp_df[["dt_iso", "ghi_cloudy_sky"]].copy()

temp_df.rename(columns={temp_df.columns[0]: 'timestamp'}, inplace=True)
temp_df['timestamp'] = temp_df['timestamp'].astype(str).str.replace(" +0000 UTC", "", regex=False)
temp_df['timestamp'] = pd.to_datetime(temp_df['timestamp']).dt.round('h')
temp_df.rename(columns={temp_df.columns[1]: 'ghilevels'}, inplace=True)
temp_list.append(temp_df)

df_temp = pd.concat(temp_list, ignore_index=True)
df = pd.merge(df, df_temp, on='timestamp', how='left')
df['ghilevels'] = df['ghilevels'].ffill()

sunrisesetlist = []
for file in ["San Antonio, Texas 2004-01-01 to 2005-12-31 (1).xlsx", "San Antonio, Texas 2006-01-01 to 2008-12-31.xlsx", "San Antonio, Texas 2009-01-01 to 2012-12-31.xlsx", "San Antonio, Texas 2013-01-01 to 2016-12-31.xlsx", "San Antonio, Texas 2017-01-01 to 2021-12-31.xlsx", "San Antonio, Texas 2022-01-01 to 2024-12-31.xlsx"]:
    df_sun = pd.read_excel(file)
    df_sun['sunrise'] = pd.to_datetime(df_sun['sunrise']).dt.round('h')
    df_sun['sunset'] = pd.to_datetime(df_sun['sunset']).dt.round('h')
    sunrisesetlist.append(df_sun)


sun_df = pd.concat(sunrisesetlist, ignore_index=True)
df = df.merge(sun_df[['sunrise','sunset']], left_on=df['timestamp'].dt.date, right_on=sun_df['sunrise'].dt.date, how='left')
df['is_daylight'] = (df['timestamp'] >= df['sunrise']) & (df['timestamp'] <= df['sunset'])

# drop extra columns
df = df.drop(columns=['sunrise', 'sunset'])

df.head(30)

/usr/local/lib/python3.12/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/local/lib/python3.12/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/local/lib/python3.12/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/local/lib/python3.12/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/local/lib/python3.12/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default sty

Combined shape: (184083, 9)


,key_0,timestamp,temp,humidity,windspeed,cloudcover,hour,day,year,month,ghilevels,is_daylight
0,2004-01-01,2004-01-01 00:00:00,62.8,91.62,6.7,100.0,0,1,2004,1,0.00,False
1,2004-01-01,2004-01-01 01:00:00,62.9,92.30,5.0,100.0,1,1,2004,1,0.00,False
2,2004-01-01,2004-01-01 02:00:00,62.7,95.24,5.0,100.0,2,1,2004,1,0.00,False
3,2004-01-01,2004-01-01 03:00:00,62.9,93.85,6.9,100.0,3,1,2004,1,0.00,False
4,2004-01-01,2004-01-01 04:00:00,62.9,96.88,8.1,100.0,4,1,2004,1,0.00,False
5,2004-01-01,2004-01-01 05:00:00,62.9,100.00,11.6,100.0,5,1,2004,1,0.00,False
6,2004-01-01,2004-01-01 06:00:00,62.7,100.00,5.8,100.0,6,1,2004,1,0.00,False
7,2004-01-01,2004-01-01 07:00:00,64.1,95.71,4.7,100.0,7,1,2004,1,2.62,True
8,2004-01-01,2004-01-01 08:00:00,65.0,100.00,9.2,100.0,8,1,2004,1,36.07,True
9,2004-01-01,2004-01-01 09:00:00,66.4,93.95,8.9,100.0,9,1,2004,1,83.62,True


In [79]:
features = ['temp', 'humidity', 'windspeed', 'cloudcover', 'hour', 'day', 'year', 'month', 'is_daylight']

scaler_X = StandardScaler()
X = df[features]
y = df['ghilevels']

In [80]:
scaler = StandardScaler()
y_scaler = MinMaxScaler()

y_scaled = y_scaler.fit_transform(y.values.reshape(-1,1))
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)

print("Any NaN in X_train? ", np.isnan(X_train).any())
print("Any infinite in X_train? ", np.isinf(X_train).any())
print("Any NaN in y_train? ", np.isnan(y_train).any())
print("Any infinite in y_train? ", np.isinf(y_train).any())
print(f"Total NaNs in y: {y.isna().sum()} out of {len(y)} samples")

from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

model = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(256, activation='relu'),
    Dense(256, activation='relu'),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(1, activation='relu')
])

from tensorflow.keras.callbacks import ReduceLROnPlateau

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=2,
    min_lr=1e-6
)

opt = Adam(learning_rate=0.001)
model.compile(optimizer=opt, loss='mse', metrics=['mae'])
model.fit(X_train, y_train, epochs=30, batch_size=32, validation_split=0.1, callbacks=[reduce_lr], verbose=1)
                                       #^^^^^^^^^^^^

y_pred = model.predict(X_test).flatten()
eps = 1e-8
smape = 100 * np.mean(
    2 * np.abs(y_pred - y_test) / (np.abs(y_test) + np.abs(y_pred) + eps)
)
mae = np.mean(np.abs(y_pred - y_test))
rmse = np.sqrt(np.mean((y_pred - y_test)**2))

print(f"sMAPE: {smape:.2f}%")
print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")


for yt, yp in zip(y_test[:20], y_pred[:20]):
    yt_val = yt.item() if isinstance(yt, np.ndarray) else yt
    yp_val = yp.item() if isinstance(yp, np.ndarray) else yp
    print(f"y_true={yt_val}, y_pred={yp_val}, error%={(abs(yt_val-yp_val)/max(abs(yt_val),1e-8))*100:.2f}%")


Any NaN in X_train?  False
Any infinite in X_train?  False
Any NaN in y_train?  False
Any infinite in y_train?  False
Total NaNs in y: 0 out of 184083 samples
Epoch 1/2
4142/4142 ━━━━━━━━━━━━━━━━━━━━ 14s 3ms/step - loss: 0.0112 - mae: 0.0554 - val_loss: 0.0066 - val_mae: 0.0395 - learning_rate: 0.0010
Epoch 2/2
4142/4142 ━━━━━━━━━━━━━━━━━━━━ 12s 3ms/step - loss: 0.0070 - mae: 0.0420 - val_loss: 0.0071 - val_mae: 0.0447 - learning_rate: 0.0010
1151/1151 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step
sMAPE: 124.96%
MAE: 0.28
RMSE: 0.39
y_true=0.16146297948260485, y_pred=0.13968902826309204, error%=13.49%
y_true=0.10869711183371222, y_pred=0.20718927681446075, error%=90.61%
y_true=0.004911129655741322, y_pred=0.042220339179039, error%=759.69%
y_true=0.0, y_pred=0.0, error%=0.00%
y_true=0.0, y_pred=0.0, error%=0.00%
y_true=0.0, y_pred=0.0, error%=0.00%
y_true=0.9422366742443863, y_pred=0.7729941010475159, error%=17.96%
y_true=0.0, y_pred=0.0, error%=0.00%
y_true=0.0, y_pred=0.0, error%=0.00%
y_true=0.18

In [81]:
tomorrow = (pd.Timestamp.now(tz='US/Pacific').normalize().tz_convert(None)+ pd.Timedelta(days=1))
yesterday = tomorrow - pd.Timedelta(days=1)
print(f"Tomorrow's date: {tomorrow.date()}")
tomorrow_month = tomorrow.month
tomorrow_day = tomorrow.day

tomorrow_rows = []
for hour in range(24):
    # Filter the dataframe for the current hour across all years
    temp_rows = df[df['hour'] == hour]

    temp_tomorrow = temp_rows['temp'].mean() if not temp_rows.empty else None
    print(f"temp {temp_tomorrow}")
    humid_tomorrow = temp_rows['humidity'].mean() if not temp_rows.empty else None
    print(f"humidity {humid_tomorrow}")
    windspeed_tomorrow = temp_rows['windspeed'].mean() if not temp_rows.empty else None
    print(f"wind speed {windspeed_tomorrow}")
    cloudcover_tomorrow = temp_rows['cloudcover'].mean() if not temp_rows.empty else None
    print(f"cloud cover {cloudcover_tomorrow}")

    # Find the most frequent value for is_daylight for the current hour across all years
    is_daylight_mode = temp_rows['is_daylight'].mode()
    is_daylight = is_daylight_mode[0] if not is_daylight_mode.empty else None
    print(f"is daylight {is_daylight}")

    day = tomorrow_day
    month = tomorrow_month

    tomorrow_rows.append([
        temp_tomorrow, humid_tomorrow,
        windspeed_tomorrow, cloudcover_tomorrow,
        hour,
        day,
        tomorrow.year, month,
        is_daylight
    ])


features_scaled = ['temp', 'humidity', 'windspeed', 'cloudcover', 'hour', 'day', 'year', 'month', 'is_daylight']

X_tomorrow = pd.DataFrame(tomorrow_rows, columns=features_scaled)
print(X_tomorrow)
X_tomorrow_scaled = scaler.transform(X_tomorrow)

predicted_ghilevels = model.predict(X_tomorrow_scaled).flatten()
predicted_ghilevels_original = y_scaler.inverse_transform(predicted_ghilevels.reshape(-1,1)).flatten()

for hour, ghi in enumerate(predicted_ghilevels_original):
    print(f"Hour {hour:02d}: {ghi:.2f}")

Tomorrow's date: 2025-10-09
temp 66.00580106896102
humidity 71.85469169599793
wind speed 5.233137791682962
cloud cover 36.39332551166732
is daylight False
temp 64.9714769912658
humidity 74.55130491461348
wind speed 4.9484943292921395
cloud cover 39.75227480119932
is daylight False
temp 64.14098039215686
humidity 76.70256862745099
wind speed 4.660666666666667
cloud cover 44.09064052287581
is daylight False
temp 63.39429018380915
humidity 78.52655716334246
wind speed 4.369482466432017
cloud cover 47.490340242471646
is daylight False
temp 62.80548820232043
humidity 79.96540216399427
wind speed 4.182296962586364
cloud cover 49.92192673706166
is daylight False
temp 62.29765350019555
humidity 81.10316516751404
wind speed 4.010663538000261
cloud cover 51.6646330335028
is daylight False
temp 61.845756746186936
humidity 81.97980576196065
wind speed 3.862195280928171
cloud cover 52.46619736670578
is daylight False
temp 61.54317559640204
humidity 82.60113805240516
wind speed 3.8387693912136616
cl

In [83]:
import pandas as pd
import os
from datetime import datetime
from google.colab import drive

# --- Force remount if needed ---
drive.mount('/content/drive', force_remount=True)

# --- Directory and file paths ---
save_directory = '/content/drive/MyDrive'
one_time_file = os.path.join(save_directory, 'predicted_ghi.csv')

# Ensure directory exists
os.makedirs(save_directory, exist_ok=True)

# --- Create your DataFrame ---
ghi_df = pd.DataFrame({
    'hour': list(range(24)),
    'predicted_ghi': predicted_ghilevels_original
})

# --- Add date column ---
today = datetime.now()
ghi_df['date'] = today.strftime("%Y-%m-%d")

# --- Overwrite the CSV with the current run ---
ghi_df.to_csv(one_time_file, index=False)
print(f"✅ Predicted GHI saved for {today.strftime('%Y-%m-%d')} to {one_time_file}")

Mounted at /content/drive
✅ Predicted GHI saved for 2025-10-09 to /content/drive/MyDrive/predicted_ghi.csv
